In [1]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
# Load a pre-trained YOLOv8 model
model = YOLO('./models/best.pt')
tracker = DeepSort(max_age=30)
video_path = './Assignment Materials/Assignment Materials/15sec_input_720p.mp4'

In [2]:
!nvidia-smi

Mon Jun 23 23:20:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   70C    P0             16W /   50W |     119MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import sys
import easyocr
reader = easyocr.Reader(['en'], gpu=True)
# Check if the model is loaded correctly
if model is None:
    print("Error: Model not loaded.")
    sys.exit(1)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    results = model(frame, stream=True)

    print(results)

    for result in results:
        if result.boxes is None  or len(result.boxes) == 0:
            print("No detections found.")
            continue
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            conf = box.conf[0]
            cls = box.cls[0]
            cropped = frame[int(y1):int(y2), int(x1):int(x2)]
            cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
            label = f"{int(cls)} {model.names[int(cls)]} {conf:.2f}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow('YOLOv8 Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        print("Exiting video stream.")
        break

In [ ]:
import sys
# Check if the model is loaded correctly
if model is None:
    print("Error: Model not loaded.")
    sys.exit(1)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    results = model(frame, stream=True)

    detections = []

    print("Gathering detections...")
    for result in results:
        if result.boxes is None  or len(result.boxes) == 0:
            print("No detections found.")
            continue
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            cls = box.cls[0]
            conf = box.conf[0]
            detections.append(((x1, y1, w, h), conf, cls))

    cv2.imshow('YOLOv8 Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        print("Exiting video stream.")
        break

print("Detections gathered, tracking...")
